In [1]:
# Importer les bibliothèques necessaires pour le projet
import pandas as pd
!pip install scrapy -q
import os 
import json
from scrapy.linkextractors import LinkExtractor
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
import re

     |████████████████████████████████| 264 kB 5.1 MB/s 
     |████████████████████████████████| 3.1 MB 48.5 MB/s 
     |████████████████████████████████| 93 kB 2.0 MB/s 
     |████████████████████████████████| 251 kB 47.7 MB/s 
     |████████████████████████████████| 47 kB 3.6 MB/s 
     |████████████████████████████████| 4.1 MB 41.6 MB/s 
     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 74 kB 2.0 MB/s 


In [9]:
# Lire et stoquer le fichier result_hotels dans un DataFrame
df_hotels_information = pd.read_json("result_hotels.json")
url = df_hotels_information['url'].tolist()
df_hotels_information.head(3)

,location,name,url,score,description
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.html?lab...,8.0,Hotel Vert offers pastel-colored rooms with a ...
1,Mont Saint Michel,Le Saint Aubert,https://www.booking.com/hotel/fr/hotel-saint-a...,7.2,"Set amid lush greenery, just 1.2 mi from the M..."
2,Mont Saint Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.h...,7.2,The traditional Hotel De La Digue in the Basse...


In [10]:
class Scrap_booking_spider(scrapy.Spider):
    
    name = "coord_hotesl"
    start_urls = url

    def parse(self, response):
        coord = {"url": response.request.url,
                       "address": response.css('span.hp_address_subtitle.js-hp_address_subtitle.jq_tooltip::text').get(),
                       "lat&lon": response.css('a.jq_tooltip.loc_block_link_underline_fix.bui-link.show_on_map_hp_link.show_map_hp_link::attr(data-atlas-latlng)').get(),
                       "stars": len(response.css('span.b6dc9a9e69.adc357e4f1.fe621d6382').getall())
                    }
        return coord  

In [11]:
filename = "coord_hotels.json"

if filename in os.listdir('/root'):
        os.remove('/root' + filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.79 Safari/537.36',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'info_' + filename : {"format": "json"}
    }
})

process.crawl(Scrap_booking_spider)
process.start()

2022-07-30 18:15:27 [scrapy.utils.log] INFO: Scrapy 2.6.2 started (bot: scrapybot)
2022-07-30 18:15:27 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.7.13 (default, Apr 24 2022, 01:04:09) - [GCC 7.5.0], pyOpenSSL 22.0.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 37.0.4, Platform Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
2022-07-30 18:15:27 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko) Chrome/100.0.4896.79 Safari/537.36'}
2022-07-30 18:15:27 [scrapy.extensions.telnet] INFO: Telnet Password: b5702bc91b18f899
2022-07-30 18:15:27 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstat

In [17]:
info_coord_hotels = pd.read_json('info_coord_hotels.json')
info_coord_hotels.head()

,url,address,lat&lon,stars
0,https://www.booking.com/hotel/fr/mont-saint-mi...,"\nLa Caserne, 50170 Le Mont Saint Michel, Fran...","48.61424653,-1.51054502",4
1,https://www.booking.com/hotel/fr/hotel-saint-a...,"\nLa Caserne, 50170 Le Mont Saint Michel, Fran...","48.61293783,-1.51010513",3
2,https://www.booking.com/hotel/fr/de-la-digue.h...,"\nLa Caserne, 50170 Le Mont Saint Michel, Fran...","48.61688155,-1.51091784",3
3,https://www.booking.com/hotel/fr/hotel-gabriel...,"\nRoute du Mont Saint Michel, 50170 Le Mont Sa...","48.61538141,-1.51070997",3
4,https://www.booking.com/hotel/fr/le-relais-du-...,"\nLa Caserne, 50170 Le Mont Saint Michel, Fran...","48.61626270,-1.51090577",3


In [18]:
# Séparer la colonne lat&lon en latitude et longitude
info_coord_hotels[['latitude', 'longitude']] = info_coord_hotels['lat&lon'].str.split(',', expand=True)

In [19]:
# Conventir les deux colonnes en float
info_coord_hotels = info_coord_hotels.astype({"latitude": float, "longitude": float })

In [20]:
# Supprimer la colonne lat&lon
info_coord_hotels = info_coord_hotels.drop(columns=['lat&lon'])

In [22]:
# Supprimer les espaces dans les adresses
info_coord_hotels['address']=info_coord_hotels['address'].str.strip()

In [23]:
# Fusionner les deux DataFrame df_hotels_information et info_coord_hotels
df_hotels_all = df_hotels_information.merge(info_coord_hotels, how='inner', on='url')
df_hotels_all.head()

,location,name,url,score,description,address,stars,latitude,longitude
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.html?lab...,8.0,Hotel Vert offers pastel-colored rooms with a ...,"La Caserne, 50170 Le Mont Saint Michel, France",2,48.614700,-1.509617
1,Mont Saint Michel,Le Saint Aubert,https://www.booking.com/hotel/fr/hotel-saint-a...,7.2,"Set amid lush greenery, just 1.2 mi from the M...","La Caserne, 50170 Le Mont Saint Michel, France",3,48.612938,-1.510105
2,Mont Saint Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.h...,7.2,The traditional Hotel De La Digue in the Basse...,"La Caserne, 50170 Le Mont Saint Michel, France",3,48.616882,-1.510918
3,Mont Saint Michel,Le Mouton Blanc,https://www.booking.com/hotel/fr/le-mouton-bla...,7.1,"At the foot of the abbey, the Mouton Blanc Hot...","Grande Rue , 50170 Le Mont Saint Michel, France",3,48.636023,-1.509896
4,Mont Saint Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,8.0,Le Relais Du Roy is a 3-star hotel located nex...,"La Caserne, 50170 Le Mont Saint Michel, France",3,48.616263,-1.510906


In [24]:
df_hotels_all.to_csv("df_hotels_all.csv")